In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload


In [86]:
import logging
import logging.config

# parse arguments
import sys

# handle importing libraries based on config file
import importlib

##### PyPi Modules #####
# rate limit the queries on the LMS server
from ratelimiter import RateLimiter

import lmsquery

In [87]:
import epdlib
import Configuration

In [110]:
def do_exit(status=0):
    sys.exit(status)

In [146]:
def main():
    #### this could be moved into a configuration sub and passed into main as one clean dictionary
    import constants
    
    ########## CONSTANTS #########
    version = constants.version
    app_name = constants.app_name
    app_long_name = constants.app_long_name
    url = constants.url
    logging_cfg = Configuration.fullPath(constants.logging_cfg)
    default_cfg = constants.default_cfg
    user_cfg = Configuration.fullPath(constants.user_cfg)
    noartwork = Configuration.fullPath(constants.noartwork)
    
    waveshare = constants.waveshare
    layouts_file = constants.layouts
    
    keyError_fmt = 'KeyError: section [{}] is missing value: {}'
    configError_fmt = 'see section [{}] in config file {}'
    valError_fmt = ''
    
    ######### CONFIGURATION #########
    
    ##### SETUP LOGGING #####
    logging.config.fileConfig(logging_cfg)
    logger = logging.getLogger(__name__)
    
    ##### COMMAND LINE ARGUMENTS
    options = Configuration.Options(sys.argv)
    # add options to the configuration object
    # for options that override the configuration file options, add in the format: 
    # dest=[[ConfigFileSectionName]]__[[Option_Name]]
    # options that are not in the configuraton file can be added using the format:
    # dest=commandline_option_name
    
    # set logging level
    options.add_argument('-l', '--log-level', ignore_none=True, 
                         type=str, choices=['DEBUG', 'INFO', 'WARNING', 'ERROR'], 
                         dest='main__log_level', 
                         help='set logging level: DEBUG, INFO, WARNING, ERROR')

    # alternative user_cfg file
    options.add_argument('-c', '--config', type=str, required=False, 
                         dest='user_cfg', ignore_none=True, default=user_cfg,
                         help=f'use the specified configuration file; default user config: {user_cfg}')
    
    # list servers - 
    options.add_argument('-s', '--list-servers', action='store_true', 
                         dest='list_servers',
                         default=False,
                         help='list servers and players found on local network and exit')
    
    # parse the command line options
    options.parse_args()

    ##### ACTION COMMANDLINE SWITCHES #####
    if options.options.list_servers:
        logging.error('FIXME - need to addd routine to list server info')
        exit
        pass    
    
    # use an alternative user configuration file
    if 'user_cfg' in options.opts_dict:
        user_cfg = options.opts_dict['user_cfg']
    
    # read the configuration files
    config_file = Configuration.ConfigFile(default_cfg, user_cfg)    
    
    # merge the configuration files and the command line options
    config = Configuration.merge_dict(config_file.config_dict, options.nested_opts_dict)
    

        
    ##### OBJECTS #####
    logging.root.setLevel(config['main']['log_level'])
    
    # LMS Query rate limiter wrapper - allow max of `max_calls` per `period` (seconds)
    lmsQuery_ratelimit = RateLimiter(max_calls=1, period=3)
    # create lms query object
    try:
        lms = lmsquery.LMSQuery(**config['lms_server'])
        if not lms.player_id:
#             raise ValueError('missing `player_id` value in lms_server section of config file')
            raise ValueError(keyError_fmt.format('lms_server', 'player_id'))
    except TypeError as e:
#         logging.fatal(f'could not configure lms_server connection - see lms_server section in config file')
        logging.fatal(configError_fmt.format('lms_server', user_cfg))
        logging.fatal(f'Error: {e}')
        
    except ValueError as e:
        logging.fatal(e)
        logging.fatal(f'locate server and player information with:\n$ {app_name} --list-servers')
        do_exit(0)
    
    # setup EPD Display
    try:
        epd = importlib.import_module('.'.join([waveshare, config['layouts']['display']]))
    except (KeyError) as e:
        logging.fatal(keyError_fmt.format('layouts', 'display'))
        logging.fatal(configError_fmt.format('layouts', user_cfg))
        logging.fatal(e)
        do_exit(0)
    except (ModuleNotFoundError) as e:
        logging.fatal(keyError_fmt.format('layouts', 'display'))
        logging.fatal(configError_fmt.format('layouts', user_cfg))
        logging.fatal(e)
        do_exit(0)
    
    # setup layouts for displaying content
    try:
        layouts = importlib.import_module(layouts_file)
        playing_layout_format = getattr(layouts, config['layouts']['now_playing'])
        stopped_layout_format = getattr(layouts, config['layouts']['stopped'])
        splash_layout_format = getattr(layouts, config['layouts']['splash'])
    except (ModuleNotFoundError) as e:
        logging.fatal(f'could not import layouts file: {layouts_file}')
        logging.fatal(e)
        do_exit(0)
    except KeyError as e:
        logging.fatal(keyError_fmt.format('layouts', e.args[0]))
        logging.fatal(configError_fmt.format('layouts', user_cfg))
        logging.fatal(e)
        do_exit(0)
    
    # set resolution for screen
    resolution = [epd.EPD_HEIGHT, epd.EPD_WIDTH]
    # sort to put longest dimension first for landscape layout
    resolution.sort(reverse=True)
    
    playing_layout = epdlib.Layout(layout=playing_layout_format, resolution=resolution)
    
    
    splash_layout = epdlib.Layout(layout=splash_layout_format, resolution=resolution)
    splash_layout.update_contents({'app_name': app_name,
                                   'version': version,
                                   'url': url})
    
    playing_layout = epdlib.Layout(layout=playing_layout_format, resolution=resolution)
    stopped_layout = epdlib.Layout(layout=stopped_layout_format, resolution=resolution)
    
    # scren objects for managing writing to screen
    screen = epdlib.Screen()
    screen.epd = epd.EPD()
    screen.initEPD()
    
    
    ########## EXECUTION ##########
    # Show splash screen
    if config['main']['splash_screen']:
        # push the images in the layout to the screen
        screen.elements = splash_layout.blocks.values()
        # concat all the individual images
        screen.concat()
        # write the image
        screen.writeEPD()
    
    # refresh the screen when true
    refresh = False
    
    
    
    logging.info('return!')
    return 

In [149]:
c = main()

Configuration:parse_args:154:WARNING - ignoring unknown options: ['-f', '/home/pi/.local/share/jupyter/runtime/kernel-a2ac2af6-13d7-4b75-85de-edd6114d8625.json']
Screen:initEPD:167:INFO - <waveshare_epd.epd5in83.EPD object at 0xae390c90> initialized
<ipython-input-146-af67b0076302>:main:151:INFO - return!


In [137]:
c.args

('now_playingx',)